In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [2]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [3]:
class ResBlock(layers.Layer):
  def __init__(self, filter_nums, strides=1, residual_path=False):
      super(ResBlock, self).__init__()

      self.conv_1 = layers.Conv2D(filter_nums,(3,3),strides=strides,padding='same')
      self.bn_1 = layers.BatchNormalization()
      self.act_relu = layers.Activation('relu')

      self.conv_2 = layers.Conv2D(filter_nums,(3,3),strides=1,padding='same')
      self.bn_2 = layers.BatchNormalization()
      
      if strides !=1:
        self.block = Sequential()
        self.block.add(layers.Conv2D(filter_nums,(1,1),strides=strides))
      else:
        self.block = lambda x:x


  def call(self, inputs, training=None):

      x = self.conv_1(inputs)
      x = self.bn_1(x, training=training)
      x = self.act_relu(x)
      x = self.conv_2(x)
      x = self.bn_2(x,training=training)
      
      identity = self.block(inputs)
      outputs = layers.add([x,identity])
      outputs = tf.nn.relu(outputs)

      return outputs

In [4]:
class ResNet(keras.Model):
  def __init__(self,layers_dims,nums_class=10):
    super(ResNet,self).__init__()

    self.model = Sequential([layers.Conv2D(64,(3,3),strides=(1,1)),
                             layers.BatchNormalization(),
                             layers.Activation('relu'),
                             layers.MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='same')])

    self.layer_1 = self.ResNet_build(64,layers_dims[0])
    self.layer_2 = self.ResNet_build(128,layers_dims[1],strides=2)   
    self.layer_3 = self.ResNet_build(256,layers_dims[2],strides=2) 
    self.layer_4 = self.ResNet_build(512,layers_dims[3],strides=2)   
    self.avg_pool = layers.GlobalAveragePooling2D()                 
    self.fc_model = layers.Dense(nums_class)    

  def call(self, inputs, training=None):
    x = self.model(inputs)
    x = self.layer_1(x)        
    x = self.layer_2(x) 
    x = self.layer_3(x)                               
    x = self.layer_4(x) 
    x = self.avg_pool(x) 
    x = self.fc_model(x)
    return x

  def ResNet_build(self,filter_nums,block_nums,strides=1):
    build_model = Sequential()
    build_model.add(ResBlock(filter_nums,strides))
    for _ in range(1,block_nums):
      build_model.add(ResBlock(filter_nums,strides=1))
    return build_model

In [5]:
def ResNet18():
  return ResNet([2,2,2,2])

In [6]:
(x,y),(x_test,y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y,axis=1)
y_test = tf.squeeze(y_test,axis=1)
print(x.shape,y.shape)

170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3) (50000,)


In [7]:
def feature_scale(x,y):
  x = tf.cast(x,dtype=tf.float32)/255.
  y = tf.cast(y,dtype=tf.int32)
  return x,y

In [8]:
RestNet_model = ResNet18()
RestNet_model.build(input_shape=(None,32,32,3))
optimizer = optimizers.Adam(lr=1e-3)

In [9]:
data = tf.data.Dataset.from_tensor_slices((x,y))
data = data.map(feature_scale).shuffle(10000).batch(512)

data_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
data_test = data_test.map(feature_scale).batch(512) 

In [10]:
data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

(512, 32, 32, 3) (512,)


In [11]:
for i in range(10):
  for step,(x,y) in enumerate(data):
    with tf.GradientTape() as tape:
      logits = RestNet_model(x)
      y_one_hot = tf.one_hot(y,depth=10)
      loss = tf.losses.categorical_crossentropy(y_one_hot,logits,from_logits=True)
      loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss,RestNet_model.trainable_variables)
    optimizer.apply_gradients(zip(grads,RestNet_model.trainable_variables))
    
    if step %100==0:
      print(i,step,'loss:',float(loss))
  total_loss = 0
  total_num=0
  for x,y in data_test:

    logits = RestNet_model(x)
    
    prob = tf.nn.softmax(logits,axis=1)
    pred = tf.argmax(prob,axis=1)

    pred = tf.cast(pred,dtype=tf.int32)
    correct = tf.equal(pred,y)

    result = tf.reduce_sum(tf.cast(correct,dtype=tf.int32))

    total_loss += int(result)
    total_num += x.shape[0]

  acc = total_loss/total_num
  print(i,'acc:',acc)

0 0 loss: 2.298459768295288
0 acc: 0.1
1 0 loss: 2.3019182682037354
1 acc: 0.1985
2 0 loss: 2.1442809104919434
2 acc: 0.3636
3 0 loss: 1.6130566596984863
3 acc: 0.4422
4 0 loss: 1.5266038179397583
4 acc: 0.4968
5 0 loss: 1.2883946895599365
5 acc: 0.5535
6 0 loss: 1.1848487854003906
6 acc: 0.61
7 0 loss: 1.0197645425796509
7 acc: 0.6447
8 0 loss: 0.8544117212295532
8 acc: 0.6778
9 0 loss: 0.703407347202301
9 acc: 0.6783


In [12]:
total_loss = 0
total_num=0
for x,y in data_test:

  gd = RestNet_model(x)
  prob = tf.nn.softmax(gd,axis=1)
  pred = tf.argmax(prob,axis=1)
  
  pred = tf.cast(pred,dtype=tf.int32)
  correct = tf.equal(pred,y)
  
  result = tf.reduce_sum(tf.cast(correct,dtype=tf.int32))
  
  total_loss += int(result)
  total_num += x.shape[0]
  
acc = total_loss/total_num
print(acc)

0.6783
